In [1]:
import pandas as pd
import numpy as np
import math
import os
import csv
import time
from datetime import datetime
from statsmodels.tsa.tsatools import lagmat
import statsmodels.api as sm
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt

# Data processing

In [21]:
pattern_bm = '%Y-%m-%dT%H:%M:%S.%fZ'
pattern_bf = '%Y-%m-%d %H:%M:%S.%f'

dateparse_bm = lambda dates: datetime.strptime(dates, pattern_bm)
dateparse_bf = lambda dates: datetime.strptime(dates, pattern_bf)


df_trades_ltc = pd.read_csv('./trades_public/bitfinex_trades_ltc_3.csv',parse_dates=True, index_col='utc',date_parser=dateparse_bf).sort_values(by=['utc'])#.truncate(before=week_ago_str, after=now_str)
df_trades_ltc['currency'] = 'ltc'

df_trades_btc = pd.read_csv('./trades_public/bitfinex_trades_btc_3.csv',parse_dates=True, index_col='utc',date_parser=dateparse_bf).sort_values(by=['utc'])#.truncate(before=week_ago_str, after=now_str)
df_trades_btc['currency'] = 'btc'


df_trades_all = df_trades_btc.append(df_trades_ltc).sort_values(by=['utc'])





In [22]:
df_trades_all.head(5)

,timestamp,id,price,size,currency
utc,,,,,
2020-06-13 09:50:17.115,1592041817115,458426739,44.66,-2.000000,ltc
2020-06-13 09:50:33.863,1592041833863,458471052,9433.00,-0.178916,btc
2020-06-13 09:50:44.173,1592041844173,458471053,9433.00,-0.178920,btc
2020-06-13 09:50:52.807,1592041852807,458471055,9432.50,-0.012000,btc
2020-06-13 09:50:52.807,1592041852807,458471054,9432.50,-0.003000,btc


# Variables & parameters

In [4]:
# current price
pf_now_btc = df_trades_btc['price'][0] 
pf_now_ltc = df_trades_ltc['price'][0]

# list of historical prices in seconds
pf_btc = [] 
pf_ltc = []

# earlist timestamp
prev_timestamp_btc = df_trades_btc['timestamp'][0]
prev_timestamp_ltc = df_trades_ltc['timestamp'][0]

# the timestamp of when we get both trades data
prev_timestamp = prev_timestamp_btc if prev_timestamp_btc > prev_timestamp_ltc else prev_timestamp_ltc


total_bals = [] # historical balance
total_bals_hodl = [] # hold & control strategy

prices_btc = [] # list of historical prices (from trades)
prices_ltc = []

bals_ltc = []

utcs = []

timestamp_list = []

trades_my = []


#### Trading Parameters

In [5]:
premium_trade_threshold = 0.0015 #0.1%include the profit margin + transaction fee + spread


max_secs = 0

fee = 0.0003

ltc_bal_init = 0
usd_bal_init = 100000 #100k

average_size = 12
ltc_bal_min = -100
ltc_bal_max = 100
usd_bal_min = usd_bal_init * 0.5

ltc_bal = ltc_bal_init
usd_bal = usd_bal_init

#### Model Parameters

In [5]:
# model_frequency: 1 means the model is build on 1 min frequency
model_frequency = 10

### Model Parameters from ARDL

In [6]:
# load dataset
df_model = pd.read_csv('March_May_LTC_BTC_PRICES.csv',index_col = 'utc',parse_dates = True)


In [7]:
def model_ardl(df_model,frequency):
    
    # resample the 1 minute data into customized sample frequency and change it to return 
    freq = str(frequency)+'T'
    df_model_resample = df_model.resample(freq).first().pct_change()
    
    lag_btc = lagmat(df_model_resample.close_btc, maxlag = 5)
    lag_ltc = lagmat(df_model_resample.close_ltc, maxlag = 5)
    
    model_fit = ols(formula="close_ltc ~ lag_btc+lag_ltc", data=df_model_resample)
    model_fit = model_fit.fit()
    print( model_fit.summary() )
    return model_fit

In [8]:
def model_predict(pf_btc,pf_ltc,frequency, model_fit):
    exog = pd.DataFrame({'btc_price':pf_btc,'ltc_price':pf_ltc})
    
    # resample the 1 second data into customized sample frequency(based on minute) and change it to return and obtain the latest data
    exog = exog.groupby(exog.index//(60*frequency)).first().pct_change()[:-6:-1] # so the first is the latest return
#     print(exog)
    exog_lag = np.concatenate((exog['btc_price'].values,exog['ltc_price'].values))
    exog_lag = np.append (1, exog_lag)
    
    predict_ltc_return = np.inner(np.array(model_fit.params),exog_lag)   
    return predict_ltc_return

In [9]:
# ARDL return model results with 5 lags in btc & utc in the customized defined frequency
model_result = model_ardl(df_model,model_frequency)

                            OLS Regression Results                            
Dep. Variable:              close_ltc   R-squared:                       0.022
Model:                            OLS   Adj. R-squared:                  0.021
Method:                 Least Squares   F-statistic:                     19.59
Date:                Wed, 17 Jun 2020   Prob (F-statistic):           3.12e-36
Time:                        09:31:25   Log-Likelihood:                 31762.
No. Observations:                8779   AIC:                        -6.350e+04
Df Residuals:                    8768   BIC:                        -6.342e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -9.283e-06   6.94e-05     -0.134      0.8

# Simulation

In [10]:
%%time
for row in df_trades_all.itertuples():
	utc = row[0]
	timestamp = row[1]
	currency = row[5]
	price = row[3]
	size = row[4]

	if currency == 'btc':
		pf_now_btc = price
	elif currency == 'ltc':
		pf_now_ltc = price
        
		if (len(pf_btc) == 4*60*15) and (len(pf_ltc) == 4*60*15):
			# we're ready to trade!


			ltc_future_return = model_predict(pf_btc,pf_ltc,model_frequency,model_result) # model_frequency: 1 means 1 mins
			ltc_fair_price = math.exp(ltc_future_return) * pf_now_ltc

			if abs(ltc_fair_price - price)/ltc_fair_price > premium_trade_threshold:
# 				print('MAKE THE TRADE!')
# 				print(ltc_fair_price)
# 				print(price)

				if (ltc_fair_price > price) and (ltc_bal < ltc_bal_max) and (size > 0):
					# size > 0 ensures that we are only executing taker trades in out backtest
# 					print('buy')
					size_trade = min(average_size,abs(size))
					trades_my.append([utc,'buy',size_trade,price,ltc_fair_price,size_trade*price*fee])

					ltc_bal += size_trade
					usd_bal -= size_trade*price * (1 + fee) 
				elif (ltc_fair_price < price) and (ltc_bal > ltc_bal_min) and (size < 0):
					# size < 0 ensures that we are only executing taker trades in out backtest
# 					print('sell')
					size_trade = min(average_size,abs(size))
                    
					trades_my.append([utc,'sell',size_trade,price,ltc_fair_price,size_trade*price*fee])
                    
					ltc_bal -= size_trade
					usd_bal += size_trade*price * (1-fee)

				total_bal = ltc_bal*price + usd_bal
# 				print('ltc bal = ' + str(ltc_bal))
# 				print('total bal = ' + str(total_bal))
				total_bal_hodl = ltc_bal_init * price + usd_bal_init

				total_bals.append(total_bal)
				total_bals_hodl.append(total_bal_hodl)

				prices_btc.append(pf_now_btc)
				prices_ltc.append(pf_now_ltc)
				bals_ltc.append(ltc_bal)
                
				utcs.append(utc)
            






	secs_since_last_timestamp = int((timestamp - prev_timestamp)/1000)
	# secs_since_last_timestamp_btc = int((timestamp - prev_timestamp_btc)/1000)

	if secs_since_last_timestamp > max_secs:
		# just for logging purposes - the bigger this is, the less the backtest reflects actual trading conditions
		max_secs = secs_since_last_timestamp
		print ('MAX SECS = ' + str(max_secs))

	for x in range(secs_since_last_timestamp):
		pf_btc.append(pf_now_btc)
		pf_ltc.append(pf_now_ltc)

		if len(pf_ltc) > 4*60*15:
			# remove first item from list if list is too long
			pf_ltc = pf_ltc[1:]

		if len(pf_btc) > 4*60*15:
			# remove first item from list if list is too long
			pf_btc = pf_btc[1:]

		# print(timestamp)
	prev_timestamp = timestamp



MAX SECS = 16
MAX SECS = 63
MAX SECS = 77
MAX SECS = 81
MAX SECS = 124
MAX SECS = 125
CPU times: user 24.5 s, sys: 30.4 ms, total: 24.5 s
Wall time: 24.5 s


# Combine data and ploting performance 

In [11]:
df = pd.DataFrame(zip(utcs,prices_btc,prices_ltc,bals_ltc,total_bals), columns = ['utc','price_btc','price_ltc','bal_ltc','bal_total'])


df = df.set_index('utc').sort_index()

df['return_pct'] = df.bal_total.pct_change()

df['cum_return'] = (1+df['return_pct']).cumprod()



In [12]:
df.head()

,price_btc,price_ltc,bal_ltc,bal_total,return_pct,cum_return
utc,,,,,,
2020-06-15 06:35:35.945,8999.6,42.337,0.00000,100000.000000,NaN,NaN
2020-06-15 06:35:36.413,8999.6,42.338,0.75000,99999.990474,-9.526050e-08,1.000000
2020-06-15 06:36:02.282,8991.4,42.285,0.90000,99999.948821,-4.165283e-07,0.999999
2020-06-15 06:36:02.906,8991.8,42.285,6.72566,99999.874920,-7.390145e-07,0.999999
2020-06-15 06:36:20.875,8988.5,42.285,6.97566,99999.871748,-3.171379e-08,0.999999


In [13]:
# daily profit in %
df['return_pct'].resample('M').sum() * 100 

utc
2020-06-30    0.01411
Freq: M, Name: return_pct, dtype: float64

In [14]:
# daily profit in usd
df['bal_total'].diff(1).resample('D').sum()

utc
2020-06-15    14.109826
Freq: D, Name: bal_total, dtype: float64

In [15]:
%matplotlib qt
df.plot(subplots = True)
plt.title('threshold = ' + str(premium_trade_threshold) + '; Freq = ' + str(model_frequency))

Text(0.5, 1.0, 'threshold = 0.0015; Freq = 10')

# Own Trades Info

In [16]:
trades_own = pd.DataFrame(trades_my, columns = ['utc', 'side','size','price','price_fair','fee_usd'])

trades_own = trades_own.set_index('utc').sort_index()

trades_own['size_usd'] = trades_own['price'] * trades_own['size']

trades_own.head(3)

,side,size,price,price_fair,fee_usd,size_usd
utc,,,,,,
2020-06-15 06:35:36.413,buy,0.75000,42.338,42.404495,0.009526,31.753500
2020-06-15 06:36:02.282,buy,0.15000,42.285,42.353486,0.001903,6.342750
2020-06-15 06:36:02.906,buy,5.82566,42.285,42.353502,0.073901,246.338041


### buy/sell counts

In [17]:
trades_own.resample('D').side.value_counts()

utc         side
2020-06-15  buy     6
Name: side, dtype: int64

### Daily trade size & fee

In [18]:
trades_own[['size', 'size_usd','fee_usd']].resample('D').sum()

,size,size_usd,fee_usd
utc,,,
2020-06-15,25.82381,1089.763553,0.326929


In [19]:
trades_own.head()

,side,size,price,price_fair,fee_usd,size_usd
utc,,,,,,
2020-06-15 06:35:36.413,buy,0.75000,42.338,42.404495,0.009526,31.753500
2020-06-15 06:36:02.282,buy,0.15000,42.285,42.353486,0.001903,6.342750
2020-06-15 06:36:02.906,buy,5.82566,42.285,42.353502,0.073901,246.338041
2020-06-15 06:36:20.875,buy,0.25000,42.285,42.357823,0.003171,10.571250
2020-06-15 06:45:19.553,buy,6.84815,42.160,42.233036,0.086615,288.718012
2020-06-15 06:45:19.553,buy,12.00000,42.170,42.233756,0.151812,506.040000
